In [8]:
import sys
import json
import base64
import os
import pandas as pd

from urllib.request import urlopen
from urllib.request import Request
from urllib.error import URLError
from urllib.parse import urlencode
from urllib.parse import quote_plus

In [9]:
def print_json(data):
    print(json.dumps(data, sort_keys=True, indent=4, separators=(', ', ': '), ensure_ascii=False))

In [10]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

TOKEN_URL = 'https://aip.baidubce.com/oauth/2.0/token'
API_KEY = '9NVR7wXLzVQEzulHiDiUO1WF'
SECRET_KEY = 'LydgIkbuw9DcT81ZEnDlr4hWrhpeh29v'
OCR_URL = "https://aip.baidubce.com/rest/2.0/ocr/v1/idcard"

def fetch_token():
    params = {'grant_type': 'client_credentials',
              'client_id': API_KEY,
              'client_secret': SECRET_KEY}
    post_data = urlencode(params)
    post_data = post_data.encode('utf-8')
    req = Request(TOKEN_URL, post_data)
    try:
        f = urlopen(req, timeout=5)
        result_str = f.read()
    except URLError as err:
        print(err)
    result_str = result_str.decode()
    result = json.loads(result_str)

    if ('access_token' in result.keys() and 'scope' in result.keys()):
        if not 'brain_all_scope' in result['scope'].split(' '):
            print ('please ensure has check the  ability')
            exit()
        return result['access_token']
    else:
        print('please overwrite the correct API_KEY and SECRET_KEY')
        exit()
        
access_token = fetch_token()

In [11]:
import difflib
import Levenshtein

def get_equal_rate_1(str1, str2):
   return difflib.SequenceMatcher(None, str1, str2).quick_ratio()

def get_equal_rate_2(str1, str2):
   return Levenshtein.ratio(str1, str2)

if __name__ == '__main__':
    a = '123456a'
    b = '234567b'
    
    print(get_equal_rate_1(a,b))
    print(get_equal_rate_2(a,b))

0.7142857142857143
0.7142857142857143


In [12]:
df_provinces = pd.read_csv('./Postprocessing/data/provinces.csv')
df_district = pd.read_csv('./Postprocessing/data/district.csv')

areacode_unit_address = json.load(open('Postprocessing/data/areacode_unit_address.json', "r", encoding="utf-8"))

# 身份证

# 户口本

- words_result_num：识别结果数，表示words_result的元素个数
- words_result：识别结果
  - BirthAddress：出生地
  - Birthday：生日
  - CardNo：身份证号
  - Name：姓名
  - Nation：民族
  - Relationship：与户主关系
  - Sex：性别

In [19]:
# encoding:utf-8
from Postprocessing.fix_birth import fix_birthday
from Postprocessing.fix_idnum import fix_idnum
from Postprocessing.fix_birthplace import fix_birthplace

import requests
import base64

request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/household_register"
# 二进制方式打开图片文件
f = open('/home/zhoujx/OCR/证件/户口本/3.jpg', 'rb')
img = base64.b64encode(f.read())

params = {"image":img}
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
response = requests.post(request_url, data=params, headers=headers)
result = response.json()['words_result']

d = dict(zip(['BirthAddress', 'Birthday', 'CardNo', 'Name', 'Nation', 'Relationship', 'Sex'], ['出生地', '生日', '身份证号', '姓名', '民族', '与户主关系', '性别']))
for key in d:
    if key in result.keys():
        result[d[key]] = result.pop(key)['words']
    
temp1 = pd.DataFrame(result, index=[1])

# 修正出生日期
if '生日' in result and '身份证号' in result:
    result['生日'] = fix_birthday(result['生日'], result['身份证号'])
# 修正身份证号
if '生日' in result and '身份证号' in result and '出生地' in result:
    result['身份证号'] = fix_idnum(result['身份证号'], result['生日'], result['出生地'])
# 修正出生地 
    result['出生地'] = fix_birthplace(result['出生地'], result['身份证号'], df_district, areacode_unit_address)
print('\n\n')
temp2 = pd.DataFrame(result, index=[2])

temp = temp1.append(temp2)
temp.T

NameError: name 'y_m_y' is not defined

# 毕业证

In [36]:
import requests
import json
import cv2
import base64

def cv2_to_base64(image):
    data = cv2.imencode(r'/home/zhoujx/4.jpg', image)[1]
    return base64.b64encode(data.tostring()).decode('utf8')

headers = {"Content-type": "application/json"}
url = "http://192.168.63.8:18889/predict/chinese_ocr_db_crnn_mobile"

img_path = '/home/zhoujx/OCR/证件/营业执照/2447_784081ed5021d6d673f980af3269d727.jpg'
img = cv2.imread(img_path)
img_base64 = cv2_to_base64(img)
# print(img_base64)

data = {'images': [img_base64]}
r = requests.post(url=url, headers=headers, data=json.dumps(data))
# print(r.json())
result = r.json()["results"]
print(result)
result_info = result[0]['data']
for re in result_info:
    print(re['text'])

[{'data': [{'confidence': 0.5824835896492004, 'text': '一美', 'text_box_position': [[595, 40], [690, 29], [694, 60], [600, 70]]}, {'confidence': 0.9727228879928589, 'text': '营业执照', 'text_box_position': [[238, 273], [722, 270], [722, 386], [238, 389]]}, {'confidence': 0.6467512845993042, 'text': '人园外卖', 'text_box_position': [[118, 364], [195, 326], [212, 357], [135, 394]]}, {'confidence': 0.962008535861969, 'text': '注册号530902600200836', 'text_box_position': [[457, 418], [707, 418], [707, 446], [457, 446]]}, {'confidence': 0.8714941740036011, 'text': '称', 'text_box_position': [[275, 482], [310, 482], [310, 513], [275, 513]]}, {'confidence': 0.8786761164665222, 'text': '临翔区肖芳蛋糕店', 'text_box_position': [[328, 485], [501, 486], [501, 512], [328, 510]]}, {'confidence': 0.7768590450286865, 'text': '类', 'text_box_position': [[191, 538], [224, 538], [224, 572], [191, 572]]}, {'confidence': 0.9558396935462952, 'text': '型', 'text_box_position': [[274, 541], [311, 541], [311, 570], [274, 570]]}, {'c

# 营业执照识别

In [47]:
# encoding:utf-8

import requests
import base64

'''
营业执照识别
'''

request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/business_license"
# 二进制方式打开图片文件
f = open('/home/zhoujx/OCR/证件/营业执照/2200_ba42937f013ae325649776af3ef046e0.jpg', 'rb')
img = base64.b64encode(f.read())

params = {"image":img}
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
response = requests.post(request_url, data=params, headers=headers)
if response:
    print_json(response.json())

{
    "direction": 0, 
    "log_id": 1302847090958794752, 
    "words_result": {
        "单位名称": {
            "location": {
                "height": 21, 
                "left": 251, 
                "top": 544, 
                "width": 162
            }, 
            "words": "勐腊县恩家小吃店"
        }, 
        "地址": {
            "location": {
                "height": -1, 
                "left": 253, 
                "top": 642, 
                "width": 458
            }, 
            "words": "单元102室云南省西双版纳傣族自治州勐腊县金都商贸城3期4"
        }, 
        "成立日期": {
            "location": {
                "height": 21, 
                "left": 251, 
                "top": 707, 
                "width": 144
            }, 
            "words": "2016年12月13日"
        }, 
        "有效期": {
            "location": {
                "height": 0, 
                "left": -1, 
                "top": -1, 
                "width": 0
            }, 
            "words": "无"
        }, 
        "法人": {
  

# 公司API

In [76]:
import requests
import json
import cv2
import base64

def cv2_to_base64(image):
    data = cv2.imencode(r'/home/zhoujx/4.jpg', image)[1]
    return base64.b64encode(data.tostring()).decode('utf8')

headers = {"Content-type": "application/json"}
url = "http://192.168.63.8:18889/predict/chinese_ocr_db_crnn_mobile"

img_path = '/home/zhoujx/OCR/证件/营业执照/2956_2623a3d4c9e3c7c80eb43fe0f06ea275.jpg'
img = cv2.imread(img_path)
img_base64 = cv2_to_base64(img)
# print(img_base64)

data = {'images': [img_base64]}
r = requests.post(url=url, headers=headers, data=json.dumps(data))
# print(r.json())
result = r.json()["results"]
print(result)
result_info = result[0]['data']
for re in result_info:
    print(re['text'])

[{'data': [{'confidence': 0.9196857810020447, 'text': '营业执照', 'text_box_position': [[1680, 778], [2725, 778], [2725, 999], [1680, 999]]}, {'confidence': 0.9821076989173889, 'text': '统一社会信用代码', 'text_box_position': [[642, 841], [1272, 841], [1272, 922], [642, 922]]}, {'confidence': 0.8249103426933289, 'text': '扫描二维码登录', 'text_box_position': [[3381, 837], [3616, 837], [3616, 891], [3381, 891]]}, {'confidence': 0.8868769407272339, 'text': '“国家企业信用', 'text_box_position': [[3393, 886], [3612, 886], [3612, 940], [3393, 940]]}, {'confidence': 0.9446770548820496, 'text': '92532621MA6PL9PA00', 'text_box_position': [[646, 949], [1192, 949], [1192, 1017], [646, 1017]]}, {'confidence': 0.9792547821998596, 'text': '信息公示系统', 'text_box_position': [[3389, 940], [3595, 940], [3595, 981], [3389, 981]]}, {'confidence': 0.7550238370895386, 'text': '了解亚多登记、', 'text_box_position': [[3397, 976], [3607, 976], [3607, 1030], [3397, 1030]]}, {'confidence': 0.7793563604354858, 'text': 'IDGL', 'text_box_position':

# 通用文字识别（标准含位置版）

In [77]:
# encoding:utf-8

import requests
import base64

request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/general"
# 二进制方式打开图片文件
f = open('/home/zhoujx/OCR/证件/营业执照/2956_2623a3d4c9e3c7c80eb43fe0f06ea275.jpg', 'rb')
img = base64.b64encode(f.read())

params = {"image":img}
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
response = requests.post(request_url, data=params, headers=headers)
if response:
    print (response.json())
print_json(response.json())

{'words_result': [{'words': '★', 'location': {'top': 397, 'left': 2108, 'width': 45, 'height': 39}}, {'words': '★', 'location': {'top': 397, 'left': 2250, 'width': 47, 'height': 43}}, {'words': '统社会信用代码营业执照', 'location': {'top': 769, 'left': 610, 'width': 2770, 'height': 253}}, {'words': '扫描二维码登录', 'location': {'top': 848, 'left': 3386, 'width': 234, 'height': 39}}, {'words': '国家企业信用', 'location': {'top': 893, 'left': 3421, 'width': 202, 'height': 43}}, {'words': '信息公示系统', 'location': {'top': 943, 'left': 3390, 'width': 200, 'height': 37}}, {'words': '了解更多登记、', 'location': {'top': 988, 'left': 3394, 'width': 212, 'height': 41}}, {'words': '(副本)副本编号:', 'location': {'top': 1037, 'left': 2061, 'width': 600, 'height': 102}}, {'words': '备案,许可、监', 'location': {'top': 1035, 'left': 3398, 'width': 226, 'height': 37}}, {'words': '管信息', 'location': {'top': 1080, 'left': 3392, 'width': 112, 'height': 51}}, {'words': '名', 'location': {'top': 1317, 'left': 641, 'width': 86, 'height': 82}}, {'words'

# 通用文字识别（高精度含位置版）

In [72]:
# encoding:utf-8

import requests
import base64

'''
通用文字识别（高精度含位置版）
'''

request_url = "https://aip.baidubce.com/rest/2.0/ocr/v1/accurate"
# 二进制方式打开图片文件
f = open('/home/zhoujx/OCR/证件/营业执照/2517_706c3b07b27b7e68b330662c584ae59c.jpg', 'rb')
img = base64.b64encode(f.read())

params = {"image":img}
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
response = requests.post(request_url, data=params, headers=headers)
# if response:
#     print (response.json())
print_json(response.json())

{
    "log_id": 3940150611003264263, 
    "words_result": [
        {
            "location": {
                "height": 18, 
                "left": 227, 
                "top": 161, 
                "width": 133
            }, 
            "words": "统一社会信用代码"
        }, 
        {
            "location": {
                "height": 18, 
                "left": 215, 
                "top": 181, 
                "width": 128
            }, 
            "words": "92532300MA6KJ24F39"
        }, 
        {
            "location": {
                "height": 106, 
                "left": 429, 
                "top": 129, 
                "width": 545
            }, 
            "words": "营照"
        }, 
        {
            "location": {
                "height": 29, 
                "left": 520, 
                "top": 224, 
                "width": 26
            }, 
            "words": "篾"
        }, 
        {
            "location": {
                "height": 16, 
                